Here's the forum that talks about how to get MJD info from the Rubin source catalog:
https://community.lsst.org/t/how-should-one-efficiently-get-the-mjd-for-a-source-catalog/3733/4

In [31]:
# Generic python packages
import pylab as plt
import gc

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay
import lsst
from datetime import datetime

# Set a standard figure size to use
plt.rcParams['figure.figsize'] = (8.0, 8.0)
afwDisplay.setDefaultBackend('matplotlib')

In [32]:
config = 'dp02'
collections = '2.2i/runs/DP0.2'
butler = dafButler.Butler(config, collections=collections)

In [33]:
datasetType = 'calexp'
dataId = {'visit': 192350, 'detector': 175}
calexp = butler.get(datasetType, dataId=dataId)

In [42]:
# This is the time of the observation in MJD:
t1 = calexp.getInfo().getVisitInfo().getDate().get(lsst.daf.base.DateTime.MJD)
print(t1)
# You can also get UTC:
#print(calexp.getInfo().getVisitInfo().getDate().get(lsst.daf.base.DateTime.MJD,lsst.daf.base.DateTime.UTC))


59837.38009422338


Is there a way with astropy to find the difference between two times?

Is there a way to get the current date in mjd using the .now() format?

In [55]:
# Now get the time of right now and see their difference:

nt = Time.now()
#ut = Time(datetime.utcnow())



if nt.mjd - t1 > 2*365:
    print(f'Obs is older than 2 years {round(nt.mjd - t1,1)} days old')
else:
    print(f'Obs is younger than 2 years {round(nt.mjd - t1,1)} days old')

Obs is younger than 2 years 54.3 days old
